In [1]:
import csv
#from BiskupILS import iterated_local_search
#from BiskupILS import cummulative_time

In [2]:
"""
============================================================
Single-Machine Scheduling ILS 
============================================================
Author: Giovanni Cesar Meira Barboza
Date: August 2024
Description:
This script optimizes job scheduling with an Iterated Local Search algorithm. It supports 
evaluating swap and insertion moves, calculating completion times, and modifying job sequences.

============================================================
"""

# ==================================================================
#                     CONFIGURATION
# ==================================================================

import random
import time

# Define Job class

class Job:
    def __init__(self, processing_time, weight_e, weight_t):
        self.processing_time = processing_time
        self.weight_e = weight_e
        self.weight_t = weight_t

# Test values
# Define jobs from a three-column list: [processing_time, weight_e, weight_t]
job_data = [[19,7,15],[4,1,6],[15,7,6],[3,1,4],[11,5,11],[9,1,15],[12,7,7],[8,9,3],[8,8,3],[13,4,15]
    # Add more job data as needed
]

# Convert list to Job objects
jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]


# ==================================================================
#                     AUXILIARY FUNCTIONS
# ==================================================================

# Cummulative time calculation

def cummulative_time(jobs, sequence, sequence_index):
    # Calculates the sum of the processing times from the first job untill sequence_index

    current_time = 0
    for idx in range(sequence_index + 1):
        job = jobs[sequence[idx]]
        current_time += job.processing_time

    return current_time

def total_cost(jobs, sequence, common_due_date):
    A = []  # Early jobs

    current_time = 0
    for x in sequence:
        job = jobs[x]
        current_time += job.processing_time
        if current_time > common_due_date:
            break
        A.append(job)
    
    current_index = sequence.index(x)
    B = [jobs[x] for x in sequence[current_index:]]  # Late jobs

    # Cost B
    cummulative_time_B = 0
    cost_B = 0
    for job in B:
        cummulative_time_B += job.processing_time
        cost_B += cummulative_time_B * job.weight_t
        
    # Cost A
    cummulative_time_A = 0
    cost_A = 0
    A = [*reversed(A)]
    for job in A:
        cost_A += cummulative_time_A * job.weight_e
        cummulative_time_A += job.processing_time

    first_job_time = common_due_date - cummulative_time_A

    return cost_A + cost_B, first_job_time

# Completion time of a job

def completion_time(jobs, sequence, first_job_time, sequence_index):
    current_time = first_job_time
    for idx in range(sequence_index + 1):
        job = jobs[sequence[idx]]
        current_time += job.processing_time

    return current_time

# Weighted penalty calculation

def penalty(job, completion_time, common_due_date):
    if completion_time < common_due_date:
        return job.weight_e * (common_due_date - completion_time)
    else:
        return job.weight_t * (completion_time - common_due_date)

# Insertion and Swap functions

def insert_job(sequence, from_index, to_index):
    job = sequence.pop(from_index)
    if to_index > from_index:
        to_index -= 1
    sequence.insert(to_index, job)
    return sequence

def swap_job(sequence, from_index, to_index):
    # Swap the elements at from_index and to_index
    sequence[from_index], sequence[to_index] = sequence[to_index], sequence[from_index]
    return sequence

# ==================================================================
#                      EVALUATION PROCEDURE
# ==================================================================

def evaluation_procedure(jobs, sequence, common_due_date, j, insert):
    # Require: list of jobs, sequence to be explored, index of the job j to evaluate swaps and insertions possible within the sequence, insert (True -> insert; False -> swap)
    # Ensure: minimum cost of objective function, operation used (swap or insertion), index of the job swapped with or inserted before
    
    #print("evaluation", sequence, j, insert)

    F = []
    cost = total_cost(jobs, sequence, common_due_date)
    f = cost[0]
    first_job_time = cost[1]
    
    Pj = penalty(jobs[sequence[j]], completion_time(jobs, sequence, first_job_time, j), common_due_date)

    for i in range(len(sequence)):
        if i != j: # Avoid auto-insertion/auto-swap
        
            if insert and abs(i - j) != 1:   # INSERTION and avoid swap with the next/previous as an insertion
                Pi = penalty(jobs[sequence[i]], completion_time(jobs, sequence, first_job_time, i), common_due_date)
                new_sequence = sequence[:]
                new_sequence = insert_job(new_sequence, j, i)
                
                k = i           # insert before j
                if j < i: k-=1  # insert after j
            
                new_first_job_time = total_cost(jobs, new_sequence, common_due_date)[1]

                Pi_new = penalty(jobs[sequence[i]], completion_time(jobs, new_sequence, new_first_job_time, k+1), common_due_date) # Penalty value of job i after insertion
                Pj_new = penalty(jobs[sequence[j]], completion_time(jobs, new_sequence, new_first_job_time, k), common_due_date)
                
                # f' = f + (Pi' - Pi) + (Pj' - Pj)
                f_new = f + (Pi_new - Pi) + (Pj_new - Pj)
                F.append([f_new, "i", i])     # (new objective function, i for insertion, index to insert before)
    
            elif not insert:    # SWAP
                new_sequence = sequence[:]
                new_sequence = swap_job(new_sequence, j, i)

                if j > i:   # swap before j
                    l = i
                    k = j
                else:       # swap after j
                    l = j
                    k = i

                new_first_job_time = total_cost(jobs, new_sequence, common_due_date)[1]

            # f' = f + sum^j_{l=i}(Pl'-Pl)
                sum_penalty = 0
                while l <= k:
                    Pi = penalty(jobs[sequence[l]], completion_time(jobs, sequence, first_job_time, l), common_due_date)
                    Pi_new = penalty(jobs[new_sequence[l]], completion_time(jobs, new_sequence, new_first_job_time, l), common_due_date)
                    sum_penalty += (Pi_new - Pi)
                    l += 1
                f_new = f + sum_penalty
                F.append([f_new, "s", i])     # (new objective function, s for swap, index to swap with)

    return min(F, key=lambda x: x[0])

# ==================================================================
#                      LOCAL SEARCH
# ==================================================================

def local_search(jobs, sequence, common_due_date, job_index, threshold_swaps, threshold_inserts, insert_probability):
    # Require: list of jobs, sequence, index of the job in the sequence, number of jobs before and after for swaps and inserts neighborhood, probability of choosing insert 
    # Ensure: sequence with least weighted tardiness/earliness within the neighborhood after all swaps or insertions are made around the chosen job

    original_sequence = sequence[:]

    insert = random.random() >= insert_probability
    if insert:  # Insertion move
        job = sequence[job_index]
        start = max(job_index - threshold_inserts, 0)
        end = min(job_index + threshold_inserts + 1, len(sequence))
        sublist = sequence[start:end]

        sublist_job_index = sublist.index(job)
        move = evaluation_procedure(jobs, sublist, common_due_date, sublist_job_index, True)
        sublist = insert_job(sublist, sublist_job_index, move[2])

        sequence[start:end] = sublist
    
    else:   # Swap move
        job = sequence[job_index]
        start = max(job_index - threshold_swaps, 0)
        end = min(job_index + threshold_swaps + 1, len(sequence))
        sublist = sequence[start:end]

        sublist_job_index = sublist.index(job)
        move = evaluation_procedure(jobs, sublist, common_due_date, sublist_job_index, False)
        sublist = swap_job(sublist, sublist_job_index, move[2])

        sequence[start:end] = sublist
    
    if total_cost(jobs, original_sequence, common_due_date)[0] < total_cost(jobs, sequence, common_due_date)[0]:    # Check whether original solution before move is better
        return(original_sequence)
    else:
        return sequence

# ==================================================================
#                  ITERATED LOCAL SEARCH
# ==================================================================

def perturbation(tabu, n, k):
    # Require: "tabu" list with previous job indexes, lenght of sequence (n), iterations (k) for a job to leave the tabu list
    # Ensure: updated tabu list and job index suggestion for next iteration

    if len(tabu) == k:
        tabu = tabu[1:]
    
    all_indices = list(range(n))
    valid_indices = [i for i in all_indices if i not in tabu]
    if valid_indices:
        job_index = random.choice(valid_indices)
    tabu.append(job_index)

    return tabu, job_index

def iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter):
    start_time = time.time()

    job_index = random.randint(0, len(initial_sequence) - 1)    # Chooses a random index from the list to start
    iter_no_improv = 0
    best_sequence = initial_sequence[:]
    best_cost = total_cost(jobs, initial_sequence, common_due_date)[0]
    iter_count = 0
    tabu = [job_index]

    sequence = initial_sequence[:]
    while iter_no_improv < stop_iter:
        sequence = local_search(jobs, sequence, common_due_date, job_index, threshold_swaps, threshold_inserts, insert_probability)
        new_total_cost = total_cost(jobs, sequence, common_due_date)
        if new_total_cost[0] < best_cost:
            best_sequence = sequence[:]
            best_cost = new_total_cost[0]
            iter_no_improv = 0
        else:
            iter_no_improv += 1

        iter_count += 1

        # To follow the article, from this point there should be Tabu-based, Construction-based and random perturbations
        # I implemented here my own perturbation, which simply chooses a random job, avoiding to choose the same ones from the previous iterations

        tabu, job_index = perturbation(tabu, len(sequence), 10)     # k = 10 for tabu search is suggested by Qin et al. 2015 
        
        end_time = time.time()
        elapsed_time = end_time - start_time

    return best_sequence, round(best_cost, 2), iter_count, round(elapsed_time, 3)

# ==================================================================
#                       RUN MAIN FUNCTION
# ==================================================================

def main():
    initial_sequence = [4, 2, 1, 3, 7, 9, 6, 5, 8, 10]
    initial_sequence = [x - 1 for x in initial_sequence]
    common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * 0.8  # Common due date for all jobs
    threshold_swaps = len(initial_sequence)//2
    threshold_inserts = len(initial_sequence)//3
    insert_probability = 0.5
    stop_iter = 50
    ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
    print(f'new_sequence = {ils[0]}, best_cost = {ils[1]}, iter_count = {ils[2]}')
    x = total_cost(jobs,initial_sequence,common_due_date)[0]
    print(f'Melhoria = {x - ils[1]}')

if __name__ == "__main__":
    main()

new_sequence = [5, 1, 9, 3, 0, 4, 6, 2, 8, 7], best_cost = 815, iter_count = 81
Melhoria = 708


### Import Jobs

(from Biskup's benchmark)

In [3]:
# Initialize an empty list to hold the data
data = []

# Read the CSV file with semi-colon delimiter
with open('job_data_biskup.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        # Convert k and sch to integers, and store the job list as a string
        k = int(row[0])
        sch = int(row[1])
        job_list = row[2]
        data.append([k, sch, job_list])

# Print the data
for row in data:
    print(row)


[0, 10, '[[20,4,5],[6,1,15],[13,5,13],[13,2,13],[12,7,6],[12,9,8],[12,5,15],[3,6,1],[12,6,8],[13,10,1]]']
[1, 10, '[[6,5,9],[19,8,12],[20,5,1],[16,8,15],[11,3,12],[11,6,1],[5,9,13],[11,7,1],[10,10,2],[20,5,1]]']
[2, 10, '[[12,6,12],[11,4,9],[7,6,3],[20,5,2],[11,9,4],[10,4,8],[19,3,10],[20,8,2],[9,4,10],[6,9,15]]']
[3, 10, '[[19,7,15],[4,1,6],[15,7,6],[3,1,4],[11,5,11],[9,1,15],[12,7,7],[8,9,3],[8,8,3],[13,4,15]]']
[4, 10, '[[5,8,4],[16,7,3],[2,7,8],[6,7,6],[10,1,2],[8,8,8],[5,1,5],[16,10,3],[7,2,11],[19,8,15]]']
[5, 10, '[[11,9,7],[9,4,15],[9,8,11],[10,4,8],[10,8,9],[6,5,15],[18,4,2],[5,1,4],[5,6,5],[5,3,14]]']
[6, 10, '[[18,9,10],[1,10,12],[16,8,13],[13,10,10],[9,4,1],[5,2,7],[12,8,15],[13,2,5],[4,10,13],[12,8,14]]']
[7, 10, '[[4,10,14],[14,7,9],[4,9,1],[16,6,12],[9,10,7],[4,9,14],[5,2,12],[2,3,5],[3,4,6],[18,2,13]]']
[8, 10, '[[10,1,2],[6,4,6],[11,9,3],[20,3,14],[8,6,14],[3,7,7],[18,6,10],[10,4,6],[1,8,15],[5,7,13]]']
[9, 10, '[[16,2,6],[19,10,3],[18,10,6],[3,9,1],[8,2,6],[2,4,14],[1

In [4]:
# Dictionary to hold job lists grouped by sch
grouped_jobs = {}

# Process each entry in the data
for entry in data:
    k = entry[0]
    sch = entry[1]
    job_list_str = entry[2]

    # Manually parse the job list string
    job_list_str = job_list_str.strip('[]')  # Remove outer brackets
    job_list = [list(map(int, job.split(','))) for job in job_list_str.split('],[')]

    # Group job lists by sch
    if sch not in grouped_jobs:
        grouped_jobs[sch] = []
    grouped_jobs[sch].append(job_list)

In [5]:
# Define Job class

class Job:
    def __init__(self, processing_time, weight_e, weight_t):
        self.processing_time = processing_time
        self.weight_e = weight_e
        self.weight_t = weight_t

In [6]:
# Lists of jobs in order of instances, 10 elements each

jobs_10 = grouped_jobs.get(10, [0]) 
jobs_20 = grouped_jobs.get(20, [0]) 
jobs_50 = grouped_jobs.get(50, [0]) 
jobs_100 = grouped_jobs.get(100, [0]) 
jobs_200 = grouped_jobs.get(200, [0]) 
jobs_500 = grouped_jobs.get(500, [0]) 
jobs_1000 = grouped_jobs.get(1000, [0]) 

### Import initial sequences

(from the heuristics_99)

In [7]:
# Dictionary to hold the grouped data by 'sch'
grouped_data = {}

# Open the CSV file and read it
with open('result_heuristic99_fmt_fixed_v2.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    
    # Skip the first line (header)
    next(reader)
    
    # Process each row in the CSV
    for row in reader:
        sch = row[2]  # 'sch' column
        heuristic_solution = row[8].strip('"')  # 'heuristic_solution' column
        
        # Convert the heuristic_solution string to a list of integers
        solution_list = list(map(int, heuristic_solution.strip('[]').split(', ')))

        # Group the lists by 'sch'
        if sch not in grouped_data:
            grouped_data[sch] = []
        grouped_data[sch].append(solution_list)



In [8]:
# Lists of sequences, in order of instances > in order of h, 40 elements each

sequences_10 = grouped_data.get('sch10', [0])
sequences_20 = grouped_data.get('sch20', [0])
sequences_50 = grouped_data.get('sch50', [0])
sequences_100 = grouped_data.get('sch100', [0])
sequences_200 = grouped_data.get('sch200', [0])
sequences_500 = grouped_data.get('sch500', [0])
sequences_1000 = grouped_data.get('sch1000', [0])

### ILS-Biskup Heuristic run

In [15]:
job_data = jobs_10[0]
jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]

job_data

[[20, 4, 5],
 [6, 1, 15],
 [13, 5, 13],
 [13, 2, 13],
 [12, 7, 6],
 [12, 9, 8],
 [12, 5, 15],
 [3, 6, 1],
 [12, 6, 8],
 [13, 10, 1]]

In [16]:
initial_sequence = [4, 2, 1, 3, 7, 9, 6, 5, 8, 10]
initial_sequence = [x - 1 for x in initial_sequence]

common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * 0.8

threshold_swaps = 4
threshold_inserts = 4
insert_probability = 0.5
stop_iter = 100
ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
print(f'new_sequence = {ils[0]}, best_cost = {ils[1]}, iter_count = {ils[2]}, elapsed_time = {ils[3]}')
print(total_cost(jobs, initial_sequence, common_due_date))

new_sequence = [3, 1, 0, 2, 6, 8, 5, 4, 7, 9], best_cost = 902, iter_count = 100, elapsed_time = 0.009
(902, 4.800000000000011)


In [13]:
# 10 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_10[0])//2
threshold_inserts = len(sequences_10[0])//3
insert_probability = 0.5
stop_iter = 100

# Heuristic Run

solution_10 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_10[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_10[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        print(f'h = {round(h,2)}, initial_sequence = {initial_sequence}, ils = {ils}')
        if i == 0 and j == 2: print(jobs_10[i])
        solution_10.append(ils)

h = 0.2, initial_sequence = [3, 1, 7, 6, 2, 5, 8, 4, 0, 9], ils = ([3, 1, 7, 6, 2, 5, 8, 4, 0, 9], 2079, 100, 0.013)
h = 0.4, initial_sequence = [3, 1, 2, 6, 5, 8, 4, 7, 0, 9], ils = ([3, 1, 2, 6, 5, 8, 4, 7, 0, 9], 1057, 100, 0.012)
[[20, 4, 5], [6, 1, 15], [13, 5, 13], [13, 2, 13], [12, 7, 6], [12, 9, 8], [12, 5, 15], [3, 6, 1], [12, 6, 8], [13, 10, 1]]
h = 0.6, initial_sequence = [3, 1, 2, 6, 8, 5, 4, 7, 0, 9], ils = ([3, 1, 2, 6, 8, 5, 4, 7, 0, 9], 841, 100, 0.01)
h = 0.8, initial_sequence = [3, 1, 0, 2, 6, 8, 5, 4, 7, 9], ils = ([3, 1, 0, 2, 6, 8, 5, 4, 7, 9], 902, 100, 0.01)
h = 0.2, initial_sequence = [3, 0, 6, 4, 1, 8, 5, 7, 2, 9], ils = ([4, 0, 6, 3, 1, 8, 5, 7, 2, 9], 1125, 103, 0.011)
h = 0.4, initial_sequence = [4, 1, 3, 6, 0, 8, 5, 7, 2, 9], ils = ([4, 1, 3, 6, 0, 8, 5, 7, 2, 9], 615, 100, 0.019)
h = 0.6, initial_sequence = [4, 1, 3, 6, 0, 8, 5, 7, 2, 9], ils = ([4, 1, 3, 0, 8, 6, 5, 7, 2, 9], 894, 108, 0.017)
h = 0.8, initial_sequence = [4, 1, 3, 6, 0, 8, 5, 7, 2, 9], ils

h = 0.8, initial_sequence = [6, 1, 5, 8, 0, 9, 2, 4, 3, 7], ils = ([6, 1, 5, 8, 0, 4, 2, 9, 3, 7], 1159, 108, 0.015)
h = 0.2, initial_sequence = [5, 4, 1, 3, 9, 0, 6, 2, 7, 8], ils = ([5, 4, 1, 3, 9, 0, 6, 2, 7, 8], 2169, 100, 0.02)
h = 0.4, initial_sequence = [1, 9, 3, 0, 5, 4, 6, 2, 7, 8], ils = ([5, 1, 9, 3, 4, 0, 6, 2, 7, 8], 1251, 127, 0.038)
h = 0.6, initial_sequence = [5, 1, 9, 3, 0, 4, 6, 2, 7, 8], ils = ([5, 1, 9, 3, 0, 4, 6, 2, 7, 8], 815, 100, 0.03)
h = 0.8, initial_sequence = [5, 1, 9, 0, 4, 6, 3, 2, 7, 8], ils = ([5, 1, 9, 3, 0, 4, 6, 2, 8, 7], 815, 110, 0.178)
h = 0.2, initial_sequence = [8, 5, 2, 3, 6, 0, 9, 4, 1, 7], ils = ([8, 5, 2, 3, 6, 0, 9, 4, 1, 7], 1220, 100, 0.072)
h = 0.4, initial_sequence = [6, 8, 9, 3, 2, 5, 0, 4, 1, 7], ils = ([6, 8, 9, 3, 2, 5, 0, 4, 1, 7], 630, 100, 0.032)
h = 0.6, initial_sequence = [4, 6, 8, 9, 5, 2, 3, 0, 1, 7], ils = ([4, 6, 8, 9, 5, 2, 3, 0, 1, 7], 521, 100, 0.059)
h = 0.8, initial_sequence = [4, 6, 8, 9, 5, 2, 3, 0, 1, 7], ils = ([4,

In [55]:
solution_10

[([3, 1, 7, 6, 2, 5, 8, 4, 0, 9], 2079, 100, 0.027),
 ([3, 1, 2, 6, 5, 8, 4, 7, 0, 9], 1057, 100, 0.02),
 ([3, 1, 2, 6, 8, 5, 4, 7, 0, 9], 841, 100, 0.033),
 ([3, 1, 0, 2, 6, 8, 5, 4, 7, 9], 902, 100, 0.014),
 ([4, 0, 6, 3, 1, 8, 7, 5, 9, 2], 1125, 109, 0.032),
 ([4, 1, 3, 6, 0, 8, 5, 7, 2, 9], 615, 100, 0.017),
 ([4, 1, 3, 0, 8, 6, 5, 7, 2, 9], 894, 105, 0.014),
 ([2, 4, 1, 3, 5, 0, 8, 7, 6, 9], 1512, 132, 0.045),
 ([1, 0, 9, 8, 5, 6, 2, 4, 3, 7], 1697, 100, 0.029),
 ([6, 1, 0, 9, 8, 5, 2, 4, 3, 7], 931, 100, 0.014),
 ([6, 1, 5, 8, 0, 9, 4, 3, 2, 7], 844, 108, 0.026),
 ([6, 1, 5, 8, 0, 2, 4, 9, 3, 7], 1162, 106, 0.017),
 ([5, 4, 1, 3, 9, 0, 6, 2, 7, 8], 2169, 100, 0.013),
 ([5, 1, 9, 3, 4, 0, 6, 2, 7, 8], 1251, 136, 0.044),
 ([5, 1, 9, 3, 0, 4, 6, 2, 7, 8], 815, 100, 0.024),
 ([5, 1, 9, 3, 0, 4, 6, 2, 8, 7], 815, 131, 0.019),
 ([8, 5, 2, 3, 6, 0, 9, 4, 1, 7], 1220, 100, 0.013),
 ([6, 8, 9, 3, 2, 5, 0, 4, 1, 7], 630, 100, 0.025),
 ([4, 6, 8, 9, 5, 2, 3, 0, 1, 7], 521, 100, 0.016),
 ([4

In [56]:
# 20 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_20[0])//2
threshold_inserts = len(sequences_20[0])//3
insert_probability = 0.5
stop_iter = 50

# Heuristic Run

solution_20 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_20[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_20[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        solution_20.append(ils)

In [57]:
# 50 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_50[0])//2
threshold_inserts = len(sequences_50[0])//3
insert_probability = 0.5
stop_iter = 50

# Heuristic Run

solution_50 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_50[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_50[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        solution_50.append(ils)

KeyboardInterrupt: 

Note: for more than 100 jobs I'd recommend shortening the neighborhood (lowering thresholds), otherwise it would take about an hour to run 200 jobs and a day to run 1000 jobs

In [ ]:
# 100 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_100[0])//2
threshold_inserts = len(sequences_100[0])//3
insert_probability = 0.5
stop_iter = 50

# Heuristic Run

solution_100 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_100[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_100[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        solution_100.append(ils)

In [ ]:
# 200 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_200[0])//2
threshold_inserts = len(sequences_200[0])//3
insert_probability = 0.5
stop_iter = 50

# Heuristic Run

solution_200 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_200[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_200[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        solution_200.append(ils)

In [ ]:
# 500 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_500[0])//2*2
threshold_inserts = len(sequences_500[0])//3*2
insert_probability = 0.5
stop_iter = 50

# Heuristic Run

solution_500 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_500[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_500[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        solution_500.append(ils)

KeyboardInterrupt: 

In [ ]:
# 1000 jobs

# Parameters (formulated according to Qin et al. 2015)

threshold_swaps = len(sequences_1000[0])//2
threshold_inserts = len(sequences_1000[0])//3
insert_probability = 0.5
stop_iter = 50

# Heuristic Run

solution_1000 = []

for i in range(10):             # for the 10 instances
    
    job_data = jobs_1000[i]
    jobs = [Job(processing_time, weight_e, weight_t) for processing_time, weight_e, weight_t in job_data]
   
    for j in range(1, 5, 1):    # for h = 0.2, 0.4, 0.6, 0.8
        h = j * 0.2
        initial_sequence = sequences_10[4*i + j - 1]
        initial_sequence = [x - 1 for x in initial_sequence]
        common_due_date = cummulative_time(jobs, initial_sequence, len(initial_sequence)-1) * h
        ils = iterated_local_search(jobs, common_due_date, initial_sequence, threshold_swaps, threshold_inserts, insert_probability, stop_iter)
        solution_10.append(ils)

### Organized solution to csv

In [ ]:
solution_100 = []  # just for test (because I only ran for 10 and 20 jobs), remove after run
solution_1000 = [] # just for test, remove after run
solution_500 = [] # just for test, remove after run
solution_200 = [] # just for test, remove after run
solution_50 = [] # just for test, remove after run

solution = solution_10 + solution_20 + solution_50 + solution_100 + solution_200 + solution_500 + solution_1000

In [ ]:
solution

[([1, 3, 6, 2, 5, 8, 4, 7, 0, 9], 2153, 107, 0.019),
 ([1, 3, 6, 2, 5, 8, 7, 4, 0, 9], 1069, 153, 0.039),
 ([3, 1, 0, 2, 6, 8, 5, 4, 9, 7], 912, 84, 0.009),
 ([1, 0, 3, 2, 6, 8, 5, 4, 7, 9], 915, 107, 0.011),
 ([4, 0, 6, 3, 1, 8, 7, 5, 9, 2], 1125, 144, 0.016),
 ([4, 1, 3, 6, 0, 8, 7, 9, 5, 2], 624, 189, 0.016),
 ([2, 4, 1, 3, 0, 6, 9, 8, 5, 7], 925, 89, 0.006),
 ([2, 4, 5, 3, 1, 8, 6, 0, 9, 7], 1630, 75, 0.007),
 ([5, 8, 9, 1, 0, 6, 2, 4, 7, 3], 1626, 111, 0.005),
 ([6, 5, 0, 8, 9, 1, 3, 4, 2, 7], 1055, 103, 0.008),
 ([1, 6, 5, 0, 9, 8, 4, 2, 7, 3], 904, 120, 0.009),
 ([6, 3, 1, 8, 5, 0, 9, 7, 4, 2], 1090, 112, 0.007),
 ([1, 9, 3, 5, 4, 0, 6, 2, 7, 8], 2189, 126, 0.008),
 ([5, 1, 9, 3, 4, 0, 6, 8, 2, 7], 1254, 73, 0.004),
 ([5, 1, 0, 9, 3, 4, 6, 2, 7, 8], 832, 131, 0.01),
 ([5, 1, 3, 9, 0, 4, 6, 2, 7, 8], 816, 102, 0.005),
 ([6, 8, 3, 2, 5, 9, 0, 4, 7, 1], 1188, 113, 0.007),
 ([6, 8, 9, 3, 2, 0, 5, 7, 1, 4], 642, 86, 0.005),
 ([4, 8, 6, 9, 3, 5, 0, 2, 1, 7], 584, 84, 0.004),
 ([6, 4, 

In [ ]:
# Define the header
header = ['sequence', 'cost', 'iter_count', 'elapsed_time']

# Specify the filename
filename = "ILSBiskup_results.csv"

# Open the file in write mode
with open(filename, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csvwriter = csv.writer(csvfile)
    
    # Write the header
    csvwriter.writerow(header)
    
    # Write the data
    for item in solution:
        # Flatten the sequence list to a string
        sequence = item[0]
        cost = item[1]
        iter_count = item[2]
        elapsed_time = item[3]
        
        # Write the row to the CSV file
        csvwriter.writerow([sequence, cost, iter_count, elapsed_time])